<a href="https://colab.research.google.com/github/HanselWilfred/WeatherWise-Hansel-Wilfred/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install requests matplotlib pyinputplus





## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather

# Add any other setup code here

## 🌤️ Weather Data Functions

In [ ]:
# ==== WeatherWise: Assignment-Style Implementations (Colab) ====
from typing import Optional, Dict, Any
import re, requests
from datetime import datetime

# -- tiny helpers --
def _fmt_date(d: str) -> str:
    try: return datetime.strptime(d, "%Y-%m-%d").strftime("%d %b %Y")
    except Exception: return d

def _emoji_for_temp(t: Optional[float]) -> str:
    try: t = float(t)
    except (TypeError, ValueError): return ""
    return "🔥" if t >= 35 else ("🙂" if t >= 15 else "🥶")

# 1) Data Fetching
def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    """Return {'location','current_temp','forecast':[{'date','max_temp','min_temp','rain_chance'},...]}; {} on error."""
    try:
        days = max(1, min(5, int(forecast_days or 5)))
    except Exception:
        days = 5
    if not location: return {}
    loc = re.sub(r"\b(tomorrow|today|day after|in\s+\d+\s+days)\b", "", str(location), flags=re.I)
    loc = re.sub(r"\s+", " ", loc).strip()
    if not loc: return {}
    try:
        g = requests.get("https://geocoding-api.open-meteo.com/v1/search",
                         params={"name": loc, "count": 1, "language": "en"}, timeout=10)
        g.raise_for_status()
        hits = (g.json().get("results") or [])
        if not hits: return {}
        lat, lon = hits[0]["latitude"], hits[0]["longitude"]
        city = hits[0].get("name") or loc
        f = requests.get("https://api.open-meteo.com/v1/forecast",
                         params={"latitude": lat, "longitude": lon, "timezone": "auto",
                                 "current_weather": True,
                                 "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_max"},
                         timeout=10)
        f.raise_for_status()
        data = f.json()
        daily = data.get("daily") or {}
        dates = daily.get("time") or []
        if not dates: return {}
        tmax = daily.get("temperature_2m_max") or []
        tmin = daily.get("temperature_2m_min") or []
        rain = daily.get("precipitation_probability_max") or []
        n = min(days, len(dates))
        fc = []
        for i in range(n):
            fc.append({
                "date": dates[i],
                "max_temp": tmax[i] if i < len(tmax) else None,
                "min_temp": tmin[i] if i < len(tmin) else None,
                "rain_chance": rain[i] if i < len(rain) else None,
            })
        return {"location": city,
                "current_temp": (data.get("current_weather") or {}).get("temperature"),
                "forecast": fc}
    except requests.RequestException:
        return {}





## 📊 Visualisation Functions

In [ ]:
# 1

def create_temperature_board_visualisation(weather_data: dict, output_type: str = 'display'):
    if not weather_data or not weather_data.get("forecast"):
        print("No data to visualise."); return None
    import matplotlib.pyplot as plt
    from datetime import datetime
    import calendar

    fc = weather_data["forecast"]

    # Build rows (label, lo, hi)
    rows = []
    for i, r in enumerate(fc):
        d = r.get("date","")
        try:
            dt = datetime.strptime(d, "%Y-%m-%d")
            lab = "Today" if i == 0 else calendar.day_abbr[dt.weekday()]
            label = f"{lab}  {dt.strftime('%d %b')}"
        except Exception:
            label = d or f"Day {i+1}"

        lo, hi = r.get("min_temp"), r.get("max_temp")
        if isinstance(lo,(int,float)) and isinstance(hi,(int,float)):
            if hi < lo: lo, hi = hi, lo
            rows.append((label, lo, hi))

    if not rows:
        print("No valid temperature values to plot."); return None

    labels = [x[0] for x in rows][::-1]
    lo     = [x[1] for x in rows][::-1]
    hi     = [x[2] for x in rows][::-1]
    width  = [b-a for a,b in zip(lo,hi)]
    ys     = list(range(len(rows)))[::-1]

    xmin, xmax = min(lo), max(hi)
    pad = max(1, (xmax - xmin) * 0.08)
    xmin, xmax = xmin - pad, xmax + pad

    fig = plt.figure(figsize=(9, 5))
    ax = plt.gca()

    # rounded bars
    ax.barh(ys, width, left=lo, height=0.55, align="center")

    # min/max text
    for y, a, b in zip(ys, lo, hi):
        ax.text(a - pad*0.05, y, f"{int(round(a))}°", va="center", ha="right")
        ax.text(b + pad*0.05, y, f"{int(round(b))}°", va="center", ha="left")

    ax.set_yticks(ys); ax.set_yticklabels(labels)
    ax.set_xlim(xmin, xmax)
    ax.set_xlabel("Temperature (°C)")
    ax.set_title(f"{weather_data.get('location','Location')} — Next Days")
    ax.grid(axis="x", linestyle="--", alpha=0.35)

    # cleaner look
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.tight_layout()
    if output_type == 'figure': return fig
    plt.show(); return None






In [ ]:
# 2

def create_rain_timeseries_visualisation(weather_data: dict, output_type: str = 'display'):
    if not weather_data or not weather_data.get("forecast"):
        print("No data to visualise."); return None
    import matplotlib.pyplot as plt
    from datetime import datetime
    import numpy as np

    fc = weather_data["forecast"]
    xs = np.arange(len(fc))
    # labels like 01 Nov, 02 Nov …
    def fmt(d):
        try: return datetime.strptime(d, "%Y-%m-%d").strftime("%d %b")
        except: return d
    dates = [fmt(r.get("date","")) for r in fc]

    rain = []
    for r in fc:
        v = r.get("rain_chance")
        try: v = float(v)
        except (TypeError, ValueError): v = 0.0
        rain.append(max(0.0, min(100.0, v)))
    rain = np.array(rain, dtype=float)

    # simple smoothing (moving average, window=3) to echo the black curve look
    if len(rain) >= 3:
        kernel = np.ones(3)/3.0
        smooth = np.convolve(rain, kernel, mode="same")
    else:
        smooth = rain.copy()

    fig = plt.figure(figsize=(9, 4.2))
    ax = plt.gca()

    # bars
    ax.bar(xs, rain, align="center", alpha=0.35)

    # line on top (smoothed)
    ax.plot(xs, smooth, linewidth=2)

    # labels above bars
    for x, v in zip(xs, rain):
        ax.text(x, v + 2, f"{int(v)}%", ha="center", fontsize=9)

    ax.set_title(f"Rain Chance — {weather_data.get('location','Location')}")
    ax.set_ylabel("Rain Chance (%)"); ax.set_xlabel("Day")
    ax.set_ylim(0, 100)
    ax.set_xticks(xs); ax.set_xticklabels(dates, rotation=0)
    ax.grid(axis="y", linestyle="--", alpha=0.4)

    # cleaner frame
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.tight_layout()
    if output_type == 'figure': return fig
    plt.show(); return None


## 🤖 Natural Language Processing

In [ ]:
# === Section 2: NLP + Response (with formatted dates) ===
import re
from typing import Optional
from datetime import datetime

# ---- tiny helpers ----
def format_date(d: str) -> str:
    """'YYYY-MM-DD' -> 'DD Mon YYYY' (fallback to original if parsing fails)."""
    try:
        return datetime.strptime(d, "%Y-%m-%d").strftime("%d %b %Y")
    except Exception:
        return d

def _as_float(x):
    """Safely convert to float; return None if not possible."""
    try:
        return float(x)
    except (TypeError, ValueError):
        return None

def _clean_location(text: str) -> Optional[str]:
    """Remove time words and extra spaces so geocoder sees only the place."""
    if not text:
        return None
    text = re.sub(r"\b(tomorrow|today|day after tomorrow|day after|in\s+\d+\s+day[s]?)\b", "", text, flags=re.I)
    text = re.sub(r"\s+", " ", text).strip()
    return text.title() if text else None

# ---- parsing ----
def parse_weather_question(q: str):
    """
    Returns: {"location": str|None, "day": 0..4, "attribute": "rain"|"temperature"|"general"}
    """
    q = (q or "").lower().strip()

    # Day: today=0, tomorrow=1, day after=2, or "in N day(s)" clamped 0..4
    if "day after tomorrow" in q or "day after" in q:
        day = 2
    elif "tomorrow" in q:
        day = 1
    else:
        day = 0
    m = re.search(r"\bin\s+(\d+)\s+day(s)?\b", q)
    if m:
        try:
            day = max(0, min(4, int(m.group(1))))
        except Exception:
            pass

    # Attribute intent
    if any(w in q for w in ["rain", "rainy", "umbrella", "drizzle", "shower", "showers"]):
        attr = "rain"
    elif any(w in q for w in ["temp", "temperature", "hot", "cold", "warm", "cool", "chill"]):
        attr = "temperature"
    else:
        attr = "general"

    # Location: PIN/ZIP first
    pin = re.search(r"\b\d{3,6}\b", q)
    if pin:
        return {"location": pin.group(0), "day": day, "attribute": attr}

    # Or words after "in ..." (stop before punctuation/time words)
    m = re.search(
        r"\bin\s+([a-z0-9\s\-]+?)(?=(?:\?|\.|,|$|\s+tomorrow|\s+today|\s+day after(?:\s+tomorrow)?|\s+in\s+\d+\s+day[s]?))",
        q
    )
    loc = _clean_location(m.group(1)) if m else None
    return {"location": loc, "day": day, "attribute": attr}

# ---- response generation ----
def generate_weather_response(user_question: str, fetch_fn):
    """
    fetch_fn(place) -> {"error": "..."} OR {"location": str, "forecast": [{date,max_temp,min_temp,rain_chance}, ...]}
    Returns a short natural-language sentence tailored to rain / temperature / general.
    """
    info = parse_weather_question(user_question)
    place, day, attr = info["location"], info["day"], info["attribute"]

    if not place:
        return "Please tell me a city or PIN/ZIP (e.g., 'weather in Perth')."

    data = fetch_fn(place)
    if not data or "error" in data:
        return data.get("error", "Sorry, I couldn't get the weather right now.")

    fc = data.get("forecast") or []
    if not fc:
        return "No forecast available right now."

    # Choose the requested day (clamped)
    i = min(day, len(fc) - 1)
    row = fc[i]
    city = data.get("location", place)
    date = format_date(row.get("date", "that day"))

    mx, mn = row.get("max_temp"), row.get("min_temp")
    rc = _as_float(row.get("rain_chance"))
    t = mx if mx is not None else mn

    # 35+ hot, 15..34 cool, <15 cold
    def temp_phrase(x):
        if x is None:
            return "temperature unavailable"
        return "hot and sunny" if x >= 35 else ("cool and pleasant" if x >= 15 else "cold")

    def rain_only(v):
        if v is None:
            return "No rain info."
        return ("Yes, likely — rain %d%%." % int(v)) if v >= 50 else \
               ("Possible — rain %d%%." % int(v)) if v >= 20 else \
               ("Unlikely — rain %d%%." % int(v))

    # Intent-specific replies
    if attr == "rain":
        return f"In {city} on {date}: {rain_only(rc)}"

    # temperature / general
    msg = f"In {city} on {date}: {temp_phrase(t)}"
    if t is not None:
        msg += f" (around {round(t)}°C)"
    if rc is not None and rc >= 30:
        # Only mention rain if it's notable
        msg += f", rain {int(rc)}%."
    else:
        msg += "."
    return msg




## 🧭 User Interface

In [ ]:
# === UI (A): helpers & visualisations ===

def print_forecast(result: dict):
    if not result or "error" in result:
        print(f"⚠️ {result.get('error','No data')}")
        return

    cur = result.get('current_temp')
    cur_txt = f"{cur}°C" if isinstance(cur, (int, float)) else "N/A"
    print(f"📍 {result['location']} | 🌡️ {cur_txt}")

    for f in result.get("forecast", []):
        # formatted date
        fd = format_date(f.get('date', ''))
        # pick temp + emoji
        pick = f.get("max_temp")
        if pick is None: pick = f.get("min_temp")
        emoji = "🔥" if (isinstance(pick, (int, float)) and pick >= 35) else \
                ("🙂" if (isinstance(pick, (int, float)) and pick >= 15) else "🥶")

        # temp string
        if isinstance(f.get("min_temp"), (int, float)) and isinstance(f.get("max_temp"), (int, float)):
            temp_txt = f"{f['min_temp']:.1f}–{f['max_temp']:.1f}°C"
        elif isinstance(f.get("max_temp"), (int, float)):
            temp_txt = f"{f['max_temp']:.1f}°C"
        elif isinstance(f.get("min_temp"), (int, float)):
            temp_txt = f"{f['min_temp']:.1f}°C"
        else:
            temp_txt = "temp N/A"

        # optional rain note (only if meaningful ≥30%)
        rc = f.get("rain_chance")
        try:
            rcv = float(rc)
        except (TypeError, ValueError):
            rcv = None
        rain_note = f" (rain {int(rcv)}%)" if (rcv is not None and rcv >= 30) else ""

        print(f"{fd}: {temp_txt} {emoji}{rain_note}")

def create_temperature_visualisation(result: dict):
    """Shaded min–max band + lines."""
    if not result or "error" in result or not result.get("forecast"):
        print("No data to visualise."); return
    try:
        import matplotlib.pyplot as plt
    except Exception:
        print("📉 Matplotlib not available."); return

    fc = result["forecast"]
    xs = list(range(len(fc)))
    dates = [format_date(r.get("date","")) for r in fc]
    tmin  = [r.get("min_temp") for r in fc]
    tmax  = [r.get("max_temp") for r in fc]

    plt.figure(figsize=(8,4.5))
    plt.fill_between(xs, tmin, tmax, alpha=0.25, label="Daily range")
    plt.plot(xs, tmax, marker="o", label="Max (°C)")
    plt.plot(xs, tmin, marker="o", label="Min (°C)")
    plt.title(f"Temperature (Min–Max) — {result.get('location','Location')}")
    plt.xlabel("Day"); plt.ylabel("°C"); plt.xticks(xs, dates)
    plt.grid(True, linestyle="--", alpha=0.5); plt.legend(); plt.tight_layout(); plt.show()

def create_precipitation_visualisation(result: dict):
    """Rain chance raincloud-like (stems + dots + % labels)."""
    if not result or "error" in result or not result.get("forecast"):
        print("No data to visualise."); return
    try:
        import matplotlib.pyplot as plt
    except Exception:
        print("📉 Matplotlib not available."); return

    fc = result["forecast"]
    xs = list(range(len(fc)))
    dates = [format_date(r.get("date","")) for r in fc]

    rainp = []
    for r in fc:
        v = r.get("rain_chance")
        try:
            v = float(v)
        except (TypeError, ValueError):
            v = 0.0
        rainp.append(max(0.0, min(100.0, v)))

    plt.figure(figsize=(8,4.5))
    # soft fill + line for a raincloud vibe
    import numpy as np
    plt.fill_between(xs, np.zeros(len(xs)), rainp, alpha=0.15)
    plt.plot(xs, rainp, linewidth=2)

    for x, v in zip(xs, rainp):
        plt.vlines(x, 0, v, linewidth=2)
        plt.scatter(x, v, s=60, zorder=3)
        plt.text(x, v + 2, f"{int(v)}%", ha="center", fontsize=8)

    plt.title(f"Rain Chance — {result.get('location','Location')}")
    plt.xlabel("Day"); plt.ylabel("Rain Chance (%)"); plt.xticks(xs, dates)
    plt.ylim(0, 100); plt.grid(axis="y", linestyle="--", alpha=0.5); plt.tight_layout(); plt.show()

def create_both_visualisations(result: dict):
    """Show temperature chart, then rain chart."""
    create_temperature_visualisation(result)
    create_precipitation_visualisation(result)





In [ ]:
# === UI (B): main menu loop ===

def run_app():
    print("🌦️ Welcome to WeatherWise 🌦️")
    print("👉 You can enter either a city name OR a PIN/ZIP code.")
    while True:
        print("\nPlease select one of the following:")
        print("1. View Forecast")
        print("2. Ask a Question")
        print("3. Visualise Data")
        print("4. Exit")
        choice = input("> ").strip()

        if choice == "1":
            # View Forecast
            place = input("Enter a city or PIN/ZIP: ").strip()
            result = get_weather_data(place)
            print_forecast(result)

        elif choice == "2":
            # Ask a Question (don’t ask for city again if already inside the question)
            q = input("Ask about the weather: ").strip()  # e.g., "Will it rain tomorrow in Adelaide?"

            info = parse_weather_question(q)
            place = info.get("location")

            if not place:
                # Only ask if the parser didn't find a location
                place = input("City or PIN/ZIP: ").strip()
                q_full = f"{q} in {place}"
            else:
                q_full = q  # already contains a location

            print(generate_weather_response(q_full, get_weather_data))

        elif choice == "3":
            # Visualise Data
            place = input("Enter a city to visualise: ").strip()
            result = get_weather_data(place)
            if not result or "error" in result:
                print(f"⚠️ {result.get('error','No data')}")
                continue

            # Offer Dashboard only if it's defined in your notebook
            has_dashboard = 'create_dashboard_visualisation' in globals()

            if has_dashboard:
                print("Visualise: 1) Temperature  2) Rain  3) Both  4) Dashboard")
            else:
                print("Visualise: 1) Temperature  2) Rain  3) Both")

            v = input("> ").strip().lower()

            if v in ("1", "temp", "temperature"):
                create_temperature_visualisation(result)
            elif v in ("2", "rain", "precip", "precipitation"):
                create_precipitation_visualisation(result)
            elif v in ("3", "both", "all"):
                try:
                    create_both_visualisations(result)
                except NameError:
                    # fallback if helper not defined
                    create_temperature_visualisation(result)
                    create_precipitation_visualisation(result)
            elif v in ("4", "dash", "dashboard") and has_dashboard:
                create_dashboard_visualisation(result)
            else:
                print("Choose a valid option.")

        elif choice == "4":
            print("Goodbye! ☀️")
            break

        else:
            print("Please choose 1–4.")




## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def run_app():
    print("🌦️ Welcome to WeatherWise 🌦️")
    print("👉 Enter a city (Perth) or PIN/ZIP (6000).")
    while True:
        print("\n1) View Forecast  2) Ask a Question  3) Visualise  4) Exit")
        choice = input("> ").strip()

        if choice == "1":
            place = input("Enter a city or PIN/ZIP: ").strip()
            res = get_weather_data(place)
            if not res:
                print("⚠️ No data returned. Check your internet or try another place.")
            else:
                print_forecast(res)

        elif choice == "2":
            q = input("Ask about the weather: ").strip()
            info = parse_weather_question(q)
            place = info.get("location")
            if not place:
                place = input("City or PIN/ZIP: ").strip()
                q_full = f"{q} in {place}"
            else:
                q_full = q
            print(generate_weather_response(q_full, get_weather_data))

        elif choice == "3":
            # ⬇️ paste/keep the visualise block here (shown above)
            place = input("City to visualise: ").strip()
            result = get_weather_data(place)
            if not result or "error" in result:
                print(f"⚠️ {result.get('error','No data to visualise.')}")
                continue

            print("1) Temperature  2) Rain 3) Both")
            v = input("> ").strip().lower()

            if v in ("1","temp","temperature","board"):
                create_temperature_board_visualisation(result)
            elif v in ("2","rain","timeseries"):
                create_rain_timeseries_visualisation(result)
            elif v in ("3","both","all"):
                create_temperature_visualisation(result)
                create_precipitation_visualisation(result)
            else:
                print("Choose 1, 2, or 3.")

        elif choice == "4":
            print("Goodbye! ☀️")
            break
        else:
            print("Please choose 1–4.")


## 🧪 Testing and Examples

In [ ]:
4
# Questions you can ask weatherwise
SAMPLE_QUESTIONS = [
    # Rain-only
    "Will it rain today?",
    "Is rain likely in Chennai tomorrow?",

    # Temperature-only
    "Is it cold tomorrow in Melbourne?",
    "Temperature in Adelaide?",

    # General feel
    "How is the weather?",
    "Weather in Auckland in 4 days?",
]

run_app()